In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import cdist
from skimage import measure
from matplotlib import pyplot as plt
import os
import tifffile
from tqdm import tqdm
from typing import List, Tuple

In [2]:
selected_magnification = "20x"
selected_image_type = "dw"
dw__thr = 13
max_dist = 9
aspect = np.array((1/3, 1/3))    # XY: 20x / aspect => 60x

In [3]:
dot_image_folder_path = f"/mnt/data/Imaging/202105-Deconwolf/data_210726/{selected_magnification}_{selected_image_type}"
ref_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/60x_dw"
dot_mask_folder_path = "../../data/20x_mask/dilated_labels_watershed_from60x"
ref_mask_folder_path = "../../data/60x_mask/dilated_labels_watershed"

### Read shift data

In [4]:
shifts = pd.read_csv(f"shift_correction/{selected_magnification}_{selected_image_type}.shifts.csv")
shifts.index = shifts["sid"].values
shifts.drop("sid", 1, inplace=True)

# Matching 20x_raw and reference dots

In [5]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")
dots_data = dots_data[selected_magnification == dots_data["magnification"]]
dots_data = dots_data[selected_image_type == dots_data["image_type"]]

In [6]:
matched_dots = pd.read_csv(
    os.path.join("../../data/magnifications_matching",
                 f"{selected_magnification}_{selected_image_type}.matched_dots.global_thr.tsv"
                ), sep="\t")

In [7]:
reference = pd.read_csv("../../data/60x_reference/ref__dw.global_thr.tsv", sep="\t")

In [8]:
for current_field_id in tqdm(np.unique(dots_data["sid"])):
    dot_max_z_proj = tifffile.imread(os.path.join(dot_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    ref_max_z_proj = tifffile.imread(os.path.join(ref_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    
    dot_labels = tifffile.imread(os.path.join(dot_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.from_60x.tiff")
                                ).reshape(dot_max_z_proj.shape)
    ref_labels = tifffile.imread(os.path.join(ref_mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.tiff")
                                ).reshape(ref_max_z_proj.shape)
    
    dots = dots_data.loc[current_field_id == dots_data["sid"], :].copy(
        ).sort_values("Value2", ascending=False).reset_index(drop=True)
    
    dot_coords = dots.loc[dw__thr <= dots["Value2"], ("x", "y")].copy().reset_index(drop=True)
    dot_coords2 = dot_coords.copy() / aspect
    dot_coords2["x"] += (shifts.loc[current_field_id, "x"] * 9)
    dot_coords2["y"] += (shifts.loc[current_field_id, "y"] * 9)
    ref_coords = reference.loc[reference["sid"] == current_field_id, ("x", "y")].copy().reset_index(drop=True)
    
    matched_20x_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "id_20x"].values
    matched_60x_dots = matched_dots.loc[matched_dots["series"] == current_field_id, "id_60x"].values
    max_match_dist = matched_dots.loc[matched_dots["series"] == current_field_id, "eudist"].max()

    selected_20x_dots = dot_coords.loc[matched_20x_dots, :]
    selected_20x_dots2 = dot_coords2.loc[matched_20x_dots, :]
    selected_60x_dots = ref_coords.loc[matched_60x_dots, :]

    fig3, ax = plt.subplots(figsize=(30, 10), ncols=3, constrained_layout=True)
    fig3.suptitle(f"Field #{current_field_id} (n.matched_dots={matched_20x_dots.shape[0]}; max.dist={max_match_dist:.03f})")
    print(" > Plotting dot")
    ax[0].set_title(f"{selected_magnification}_{selected_image_type} (n.total={dot_coords2.shape[0]}, only matched are plotted)")
    ax[0].imshow(
        dot_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=dot_max_z_proj.min(), vmax=dot_max_z_proj.max(),
        resample=False, filternorm=False)
    ax[0].scatter(
        x=selected_20x_dots["y"].values,
        y=selected_20x_dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting ref")
    ax[1].set_title(f"60x_dw (n.total={ref_coords.shape[0]}, only matched are plotted)")
    ax[1].imshow(
        ref_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=ref_max_z_proj.min()*1.5, vmax=ref_max_z_proj.max()*.5,
        resample=False, filternorm=False)
    ax[1].scatter(
        x=selected_60x_dots["y"].values,
        y=selected_60x_dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting contours [20x]")
    for lid in range(1, dot_labels.max()):
        contours = measure.find_contours(dot_labels == lid, 0.8)
        for contour in contours:
            ax[0].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    print(" > Plotting contours [60x]")
    for lid in range(1, ref_labels.max()):
        contours = measure.find_contours(ref_labels == lid, 0.8)
        for contour in contours:
            ax[1].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    print(" > Plotting overlapped points between raw and dw")
    ax[2].set_title(f"Red: {selected_magnification}_{selected_image_type}. Blue: 60x_dw.")
    ax[2].plot(
        selected_20x_dots2["y"].values,
        selected_20x_dots2["x"].values,
        'r.', marker=".", markersize=2)
    ax[2].plot(
        selected_60x_dots["y"].values,
        selected_60x_dots["x"].values,
        'b.', marker=".", markersize=.8)
    plt.close(fig3)
    print(" > Exporting")
    fig3.savefig(os.path.join("../../data/magnifications_matching",
                              f"{selected_magnification}_{selected_image_type}.overlays.global_thr.matched",
                              f"overlay_{current_field_id:03d}.png"), bbox_inches='tight')
    print(" ! DONE")

  0%|          | 0/7 [00:00<?, ?it/s]

 > Plotting dot
 > Plotting ref
 > Plotting contours [20x]
 > Plotting contours [60x]
 > Plotting overlapped points between raw and dw
 > Exporting


 14%|█▍        | 1/7 [00:16<01:37, 16.20s/it]

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [20x]
 > Plotting contours [60x]
 > Plotting overlapped points between raw and dw
 > Exporting


 29%|██▊       | 2/7 [00:35<01:26, 17.23s/it]

...

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [20x]
 > Plotting contours [60x]
 > Plotting overlapped points between raw and dw
 > Exporting


 86%|████████▌ | 6/7 [01:56<00:20, 20.02s/it]

 ! DONE
 > Plotting dot
 > Plotting ref
 > Plotting contours [20x]
 > Plotting contours [60x]
 > Plotting overlapped points between raw and dw
 > Exporting


100%|██████████| 7/7 [02:17<00:00, 19.65s/it]

 ! DONE
